# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** S

**Names:**

* Adam Cohen
* Stefan Peters
* Alexandre Spiess
* Tom Oliver Martin Vrakking
---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.2 Network sampling

#### Exercise 2.7: Random walk on the Facebook network

In [1]:
# Imports
import requests
import random

In [2]:
# API Setup
HOSTNAME = 'http://iccluster031.iccluster.epfl.ch'
URL_TEMPLATE = HOSTNAME + ':5050/v1.0/facebook?user={user_id}'

In [3]:
# Function to get user data
def get_user_data(user_id):

    # Follow "ix-lab2-helper" method for the API
    url = URL_TEMPLATE.format(user_id=user_id)
    response = requests.get(url)

    # Check for errors
    if response.status_code != 200:
        print(f"Error: Unable to fetch data for user {user_id}")
        return None

    # Return the data associated to the given user_id
    data = response.json()
    return data

#### Algorithm 2.1: Random walker on Facebook

**Require:** Source node `s`, number of nodes `N`

Set `u ← s` (start from the source node)

Initialize `i = 0`

**While** `i < N` **do**:
   - Get the age of node `u`
   - Select node `v` uniformly at random from the neighborhood of `u`
   - Set `u ← v` (move to the next node)
   - Increment `i ← i + 1`

**End** **while**


In [12]:
# Function for average age
def estimate_average_age(start_user, num_steps):

    # Initialize variables using parameters
    visited_users = set()
    ages = []
    current_user = start_user

    # Random Walk approach of Algorithm 2.1
    for _ in range(num_steps):

        # Get user data (check existence)
        user_data = get_user_data(current_user)
        if not user_data:
            break
        age = user_data.get("age")
        friends = user_data.get("friends", [])

        # Add age and user to list
        if age is not None and current_user not in visited_users:
            ages.append(age)
            visited_users.add(current_user)

        # Choose the user at random in the list of friends
        next_user = random.choice(friends) 
        current_user = next_user

    # Calculate and return average age
    avg_age = sum(ages) / len(ages) if ages else 0
    return avg_age, len(visited_users)

In [33]:
# Running the estimation (num_steps can be adjusted to look for optimal result)
start_user = "a5771bce93e200c36f7cd9dfd0e5deaa"
num_steps = 10000

# Retrive average age and number of distinct user visited
estimated_avg_age, users_visited = estimate_average_age(start_user, num_steps)

print(f"Estimated Average Age: {estimated_avg_age}")
print(f"Users Visited: {users_visited}")

Estimated Average Age: 21.444395149364095
Users Visited: 6762


**Some explanations:** The number of user visited will likely always be under num_steps because the algorithm regularly comes back on it's steps, when finding someone with only one friend mostly. After running the code a lot of times, the average age was always between 20 and ~24, even when changing num_steps drastically. We decided to keep a final num_step at 10'000 to be sure that we were not restrained by this parameter.

#### Exercise 2.8

Part 1: 

As we have seen in exercise 2.7, our estimated age is way under the real one published by facebook, around 20~23 against 45. This discrepancy is of the scale of 50%, so we can confidently say that following the 2.1 Algrorithm is not the most efficient way to get a correct approximate.

Part 2:

This can be caused by three major reasons :
* Small sample size (num_step is not big enough)
* Degree bias : Younger users having more friend on average than older ones so the algorithm gets attracted by younger users, failing to give a correct average.
* Graph structure bias: Facebook (like other social-networks) has age-based communities, so the random walk might take a long time to escape if starting from a specific one.

In our code we went up to a sample size of 50000 without much (any) change. So this is not the aspect we will be focusing on in part 3. 

The degree bias and graph structure bias are more likely the cause of our results. Indeed, we start with a user of 13 years old, so we are likely stuck at first and then always attracted back to young users.

Part 3:

By following the algorithm studied in class, we got a better result, since now there is almost no difference between our result and the study published by Facebook.

The algorithm consist to compensate the bias towards high-degree nodes by weighting the age with 1/degree.

The final average is computed as:
$$
\hat{F}_{RW} = \frac{\sum_{t} \frac{f(X_t)}{d(X_t)}}{\sum_{t} \frac{1}{d(X_t)}}
$$

where:
- \( f(X_t) \) is the node statistic (e.g., age) at time \( t \),
- \( d(X_t) \) is the degree of the node at time \( t \). t \).


In [4]:
# Function for average age with bias correction using weighted estimator
def estimate_average_age_wbc(start_user, num_steps):
    
    # Initialize variables using parameters
    visited_users = set()
    weighted_ages = []
    weighted_counts = []
    current_user = start_user

    # Modified random walk approach with degree weighting
    for _ in range(num_steps):
        
        # Get user data (check existence)
        user_data = get_user_data(current_user)
        if not user_data:
            break
        age = user_data.get("age")
        friends = user_data.get("friends", [])
        degree = len(friends)

        # Add weighted age contribution
        if age is not None and degree > 0:
            weighted_ages.append(age / degree)  # Normalize by degree
            weighted_counts.append(1 / degree)  # Weight for normalization

        # Choose the next user based on random walk
        current_user = random.choice(friends)
        
    # Compute weighted average to correct degree bias
    avg_age = sum(weighted_ages) / sum(weighted_counts) if weighted_counts else 0
    return avg_age

In [5]:
# Running the estimation (num_steps can be adjusted to look for optimal result)
start_user = "a5771bce93e200c36f7cd9dfd0e5deaa"
num_steps = 10000

# Retrive average age and number of distinct user visited
estimated_avg_age = estimate_average_age_wbc(start_user, num_steps)

print(f"Estimated Average Age: {estimated_avg_age}")

Estimated Average Age: 43.977264258812774
